In [1]:
import os
import scipy.io
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [2]:
sample_dir = "/Users/hugofluhr/phd_local/data/LearningHabits/data-samples"
behav_dir = os.path.join(sample_dir, "behav")

In [3]:
raw_data = scipy.io.loadmat(os.path.join(behav_dir, "habit_c07g3u_2_2024-04-30_10-35.mat"), squeeze_me=True, struct_as_record=False)

In [4]:
raw_data['phase2_1'].blocks[0].time

In [34]:
class Block:
    def __init__(self, raw_block):
        """
        Initialize a block with a list of trials.
        """
        self.raw_block = raw_block

        # these are the ITI and ISI that were used in the block, the exact timing will be in the trials themselves
        self.iti_seq = raw_block.iti_seq
        self.isi_seq = raw_block.isi_seq

        # time references
        self.scanner_trigger = raw_block.time.scanner_trigger
        self.start_time = raw_block.time.start_time
        self.end_time = raw_block.time.end_time
        self.total_length = raw_block.time.length

        self.n_trials = self.iti_seq.shape[0]
        # load the actual trials
        self.trials = self._load_trials(raw_block)

    def _load_trials(self, raw_block):
        """
        Loads the trials into a dataframe from the raw block data.
        """
        # init empty dataframe
        trials = pd.DataFrame(columns=['left_stim', 'right_stim', 'left_value', 'right_value', 'shift',
                                       'action', 'rt', 'chosen_stim', 'reward', 'correct',
                                       't_first_stim', 't_second_stim', 't_action', 't_feedback', 't_iti_onset'],
                              index=range(self.n_trials))
        # populate the dataframe
        trials.iloc[:, :5] = raw_block.seq1
        trials['action'] = raw_block.a
        trials['rt'] = raw_block.rt1
        trials['chosen_stim'] = raw_block.chosen
        # if left stim was chosen, action == 1.0
        trials['reward'] = trials.apply(lambda row: row['left_value'] if row['action'] == 1.0 else row['right_value'], axis=1)
        return trials


In [35]:
b = Block(raw_data['phase2_1'].blocks[0])
b.trials

,left_stim,right_stim,left_value,right_value,shift,action,rt,chosen_stim,reward,correct,t_first_stim,t_second_stim,t_action,t_feedback,t_iti_onset
0,2,1,2,1,0,1.0,0.682432,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,1,2,0,2.0,0.487746,3.0,2,NaN,NaN,NaN,NaN,NaN,NaN
2,6,8,4,5,0,1.0,0.465525,6.0,4,NaN,NaN,NaN,NaN,NaN,NaN
3,8,7,5,4,0,2.0,0.474767,7.0,4,NaN,NaN,NaN,NaN,NaN,NaN
4,2,4,2,3,0,1.0,0.790170,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1,3,1,2,0,2.0,0.444529,3.0,2,NaN,NaN,NaN,NaN,NaN,NaN
92,1,2,1,2,1,2.0,0.405896,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN
93,6,4,4,3,1,1.0,0.449171,6.0,4,NaN,NaN,NaN,NaN,NaN,NaN
94,7,8,4,5,1,1.0,0.397754,7.0,4,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
class Subject:
    def __init__(self, subject_data):
        self.stimuli_data = subject_data.get('stimuli')
        self.setup_data = subject_data.get('setup')
        self.training_blocks = self._load_blocks(subject_data, 'training')
        self.test_block = self._load_block(subject_data, 'test')
        self.practice_block = self._load_block(subject_data, 'practice')
        self.ratings = subject_data.get('ratings_pre')

    def _load_blocks(self, subject_data, block_type):
        """
        Loads multiple blocks from the subject data.
        """
        blocks = []
        for block_data in subject_data.get(block_type, []):
            blocks.append(self._create_block(block_data))
        return blocks

    def _load_block(self, subject_data, block_type):
        """
        Loads a single block from the subject data.
        """
        block_data = subject_data.get(block_type)
        return self._create_block(block_data) if block_data else None

    def _create_block(self, block_data):
        """
        Creates a Block instance from the block data.
        """
        trials = [Trial(trial['left_stimuli'], trial['right_stimuli'], trial['left_reward'],
                        trial['right_reward'], trial['shift'], trial['action'])
                  for trial in block_data['trials']]
        return Block(trials)

    def get_block(self, block_name):
        """
        Returns the requested block by name.
        """
        if block_name == 'training':
            return self.training_blocks
        elif block_name == 'test':
            return self.test_block
        elif block_name == 'practice':
            return self.practice_block
        else:
            raise ValueError(f"Block '{block_name}' not found.")

    def get_stimuli_data(self):
        """
        Returns the stimuli data.
        """
        return self.stimuli_data

    def get_setup_data(self):
        """
        Returns the setup data.
        """
        return self.setup_data

    def get_ratings(self):
        """
        Returns the ratings data.
        """
        return self.ratings

# Example usage:
# exp_data = ExperimentData('/path/to/your/file.mat')
# subject_data = exp_data.get_subject_data('subject_id')  # Hypothetical method to get a specific subject's data
# subject = Subject(subject_data)

# Accessing training blocks
# training_blocks = subject.get_block('training')
# for block in training_blocks:
#     trials_array = block.get_all_trials_as_array()
#     print(trials_array)

# Accessing setup data
# setup_data = subject.get_setup_data()
# print(setup_data)